# Example usage of AWSome

We will rename all the files from a bucket with a foo/bar/ prefix and copy them to another bucket.

In [16]:
from awsome import s3
from awsome.development import debug_environment, dry_run

## Intervention script

We create the function that will do the renaming/copy intervention.

In [ ]:
def intervention():
    keys = s3.ls('s3://testbucket/foo/bar/')
    for key in keys:
        new_key = key.replace('customers', 'clients')
        # Rename all objects with the same key
        s3.move_key(from_bucket='testbucket', from_key=key, to_bucket'testbucket', to_key=new_key)
        # Move all objects to the production bucket with the same key
        s3.move_key(from_bucket='testbucket', from_key=new_key, to_bucket'testbucket')

## Testing the script

Before running the script in production we should do a few tests to make sure it's doing what we think it is.

### Creating test data

We will create some dummy files that mimic the structure of the files in our real aws instance to test the intervention on them.

In [24]:
def create_test_data():
    for i in range(1, 6):
        # Files we will be changing
        key = f'foo/bar/customers_{i}.csv'
        s3.upload_string(data='some data', bucket='testbucket', key=key)
        
        # Files we want untouched
        key = f'foo/baz/companies_{i}.csv'
        s3.upload_string(data='some data', bucket='testbucket', key=key)

### Checking the test data

Let's check that the test data is created correctly. Of course we don't want to actually create those dummy files in aws. 

Instead we use the debug_environment context manager that provides a moto s3 instance where we can run our tests as they would run in a real S3 instance. We just need to pass it the name of the buckets it needs to create.

In [25]:
with debug_environment(['testbucket', 'prodbucket']):
    create_test_data()
    print(s3.ls('s3://testbucket/', recursive=True))
    #print(s3.ls('s3://prodbucket/', recursive=True))

['foo/bar/customers_1.csv', 'foo/bar/customers_2.csv', 'foo/bar/customers_3.csv', 'foo/bar/customers_4.csv', 'foo/bar/customers_5.csv', 'foo/baz/companies_1.csv', 'foo/baz/companies_2.csv', 'foo/baz/companies_3.csv', 'foo/baz/companies_4.csv', 'foo/baz/companies_5.csv']


In [ ]:
We can see that the sample data has been created correctly.